# Enable Agent Tutorial Part 4: RAG 챗봇 및 보고서 생성

## 개요

Part 3에서 구축한 지식 베이스를 활용하여 대화형 RAG 챗봇을 구현한다.

## RAG (Retrieval-Augmented Generation)

```
+------------------+
|  사용자 질문     |
+--------+---------+
         v
+-----------------------------+
|  IrisRAGChatbot             |
+-----------------------------+
| 1. 지식 베이스 검색         | <- knowledge_base.txt
| 2. 필요시 Agent 호출        | <- IrisEnableAgent
| 3. LLM으로 응답 생성        | <- OpenAI GPT-4
| 4. 컨텍스트 자동 업데이트   | -> context_store/
+-----------------------------+
         v
+------------------+
|  자연어 응답     |
+------------------+
```

## 학습 내용

- RAG 챗봇 구현
- 대화형 예측 인터페이스
- 자동 분석 보고서 생성
- 통합 데모

---
## 1. 라이브러리 임포트

In [1]:
import os
import json
import yaml
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Tuple
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
print("라이브러리 임포트 완료")

라이브러리 임포트 완료


---
## 2. IrisEnableAgent 클래스 (Part 2에서 재사용)

In [2]:
class IrisEnableAgent:
    """Iris 분류 모델을 Enable Agent로 래핑한 클래스"""
    
    def __init__(self, skill_path: str):
        with open(skill_path, 'r', encoding='utf-8') as f:
            self.skill = yaml.safe_load(f)
        
        model_path = self.skill['model_info']['model_path']
        self.model = joblib.load(model_path)
        
        metadata_path = self.skill['model_info']['metadata_path']
        with open(metadata_path, 'r', encoding='utf-8') as f:
            self.metadata = json.load(f)
        
        self.client = OpenAI()
        
        print(f"Enable Agent 초기화 완료: {self.skill['agent_name']}")
        print(f"모델 정확도: {self.metadata['accuracy']:.4f}")
    
    def get_capability_description(self) -> str:
        return f"""
Agent 이름: {self.skill['agent_name']}
버전: {self.skill['version']}
설명: {self.skill['description']}

주요 기능: {self.skill['capabilities']['primary']}
부가 기능: {', '.join(self.skill['capabilities']['secondary'])}

입력 파라미터:
- sepal_length: 꽃받침 길이 (cm, 4.0-8.0)
- sepal_width: 꽃받침 너비 (cm, 2.0-5.0)
- petal_length: 꽃잎 길이 (cm, 1.0-7.0)
- petal_width: 꽃잎 너비 (cm, 0.1-3.0)

출력:
- predicted_class: 예측된 붓꽃 품종 (setosa, versicolor, virginica)
- confidence: 예측 신뢰도 (0-1)
- probabilities: 각 클래스별 확률
- feature_importance: 특성별 중요도

모델 정보:
- 타입: {self.metadata['model_type']}
- 정확도: {self.metadata['accuracy']:.4f}
- 학습 샘플: {self.metadata['training_samples']}개
""".strip()
    
    def validate_input(self, input_data: Dict[str, float]) -> Tuple[bool, str]:
        required_fields = self.skill['input_schema']['required']
        properties = self.skill['input_schema']['properties']
        
        for field in required_fields:
            if field not in input_data:
                return False, f"필수 필드 누락: {field}"
        
        for field, value in input_data.items():
            if field in properties:
                prop = properties[field]
                if 'minimum' in prop and value < prop['minimum']:
                    return False, f"{field}가 최소값({prop['minimum']})보다 작다: {value}"
                if 'maximum' in prop and value > prop['maximum']:
                    return False, f"{field}가 최대값({prop['maximum']})보다 크다: {value}"
        
        return True, "유효한 입력"
    
    def predict(self, input_data: Dict[str, float]) -> Dict[str, Any]:
        is_valid, message = self.validate_input(input_data)
        if not is_valid:
            raise ValueError(message)
        
        feature_names = self.metadata['feature_names']
        X = np.array([[
            input_data['sepal_length'],
            input_data['sepal_width'],
            input_data['petal_length'],
            input_data['petal_width']
        ]])
        
        prediction = self.model.predict(X)[0]
        probabilities = self.model.predict_proba(X)[0]
        
        target_names = self.metadata['target_names']
        predicted_class = target_names[prediction]
        confidence = float(probabilities[prediction])
        
        feature_importance = dict(zip(
            feature_names,
            self.model.feature_importances_.tolist()
        ))
        
        return {
            "predicted_class": predicted_class,
            "confidence": confidence,
            "probabilities": dict(zip(target_names, probabilities.tolist())),
            "feature_importance": feature_importance,
            "input_features": input_data,
            "timestamp": datetime.now().isoformat()
        }
    
    def generate_tool_definition(self) -> Dict[str, Any]:
        return {
            "type": "function",
            "function": {
                "name": "predict_iris_species",
                "description": self.skill['description'],
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sepal_length": {"type": "number", "description": "꽃받침 길이 (cm)"},
                        "sepal_width": {"type": "number", "description": "꽃받침 너비 (cm)"},
                        "petal_length": {"type": "number", "description": "꽃잎 길이 (cm)"},
                        "petal_width": {"type": "number", "description": "꽃잎 너비 (cm)"}
                    },
                    "required": ["sepal_length", "sepal_width", "petal_length", "petal_width"]
                }
            }
        }

print("IrisEnableAgent 클래스 정의 완료")

IrisEnableAgent 클래스 정의 완료


---
## 3. PredictionContextBuilder 클래스 (Part 3에서 재사용)

In [3]:
class PredictionContextBuilder:
    """예측 결과를 컨텍스트로 저장하고 관리하는 클래스"""
    
    def __init__(self, context_dir: str = 'context_store'):
        self.context_dir = Path(context_dir)
        self.context_dir.mkdir(exist_ok=True)
        
        self.log_file = self.context_dir / 'prediction_logs.json'
        self.summary_file = self.context_dir / 'prediction_summary.json'
        self.knowledge_base_file = self.context_dir / 'knowledge_base.txt'
        
        self.logs = self._load_logs()
        
        print(f"Context Builder 초기화 완료")
        print(f"저장된 예측 로그: {len(self.logs)}개")
    
    def _load_logs(self) -> List[Dict[str, Any]]:
        if self.log_file.exists():
            with open(self.log_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return []
    
    def add_prediction(self, prediction_result: Dict[str, Any]):
        self.logs.append(prediction_result)
        
        with open(self.log_file, 'w', encoding='utf-8') as f:
            json.dump(self.logs, f, indent=2, ensure_ascii=False)
        
        print(f"예측 결과 저장 완료 (총 {len(self.logs)}개)")
        
        self._update_summary()
        self._update_knowledge_base()
    
    def _update_summary(self):
        if not self.logs:
            return
        
        total_predictions = len(self.logs)
        class_counts = {}
        confidence_sum = 0
        
        for log in self.logs:
            predicted_class = log['predicted_class']
            class_counts[predicted_class] = class_counts.get(predicted_class, 0) + 1
            confidence_sum += log['confidence']
        
        avg_confidence = confidence_sum / total_predictions
        
        feature_stats = {'sepal_length': [], 'sepal_width': [], 'petal_length': [], 'petal_width': []}
        for log in self.logs:
            for feature, value in log['input_features'].items():
                feature_stats[feature].append(value)
        
        feature_averages = {feature: sum(values) / len(values) for feature, values in feature_stats.items()}
        
        summary = {
            "total_predictions": total_predictions,
            "class_distribution": class_counts,
            "average_confidence": avg_confidence,
            "feature_averages": feature_averages,
            "last_updated": datetime.now().isoformat()
        }
        
        with open(self.summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
    
    def _update_knowledge_base(self):
        if not self.logs:
            return
        
        with open(self.summary_file, 'r', encoding='utf-8') as f:
            summary = json.load(f)
        
        knowledge_text = f"""# Iris 분류 모델 예측 지식 베이스

## 전체 통계 (마지막 업데이트: {summary['last_updated']})

총 예측 수행 횟수: {summary['total_predictions']}회
평균 예측 신뢰도: {summary['average_confidence']:.2%}

## 품종별 예측 분포\n\n"""
        
        for species, count in summary['class_distribution'].items():
            percentage = (count / summary['total_predictions']) * 100
            knowledge_text += f"- {species}: {count}회 ({percentage:.1f}%)\n"
        
        knowledge_text += "\n## 평균 특성 값\n\n"
        for feature, avg_value in summary['feature_averages'].items():
            knowledge_text += f"- {feature}: {avg_value:.2f} cm\n"
        
        knowledge_text += "\n## 최근 예측 기록 (최근 5개)\n\n"
        recent_logs = self.logs[-5:]
        for i, log in enumerate(reversed(recent_logs), 1):
            knowledge_text += f"### 예측 #{len(self.logs) - i + 1}\n"
            knowledge_text += f"- 예측 품종: {log['predicted_class']}\n"
            knowledge_text += f"- 신뢰도: {log['confidence']:.2%}\n\n"
        
        with open(self.knowledge_base_file, 'w', encoding='utf-8') as f:
            f.write(knowledge_text.strip())
    
    def get_knowledge_base_content(self) -> str:
        if self.knowledge_base_file.exists():
            with open(self.knowledge_base_file, 'r', encoding='utf-8') as f:
                return f.read()
        return ""
    
    def get_summary(self) -> Dict[str, Any]:
        if self.summary_file.exists():
            with open(self.summary_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}

print("PredictionContextBuilder 클래스 정의 완료")

PredictionContextBuilder 클래스 정의 완료


---
## 4. IrisRAGChatbot 클래스 구현

### 핵심 기능

- **지식 베이스 통합**: 과거 예측 결과를 컨텍스트로 활용
- **Function Calling**: 실시간 예측 수행
- **자동 컨텍스트 업데이트**: 새로운 예측을 지식 베이스에 추가
- **보고서 생성**: 전체 예측 결과 분석

In [4]:
class IrisRAGChatbot:
    """Iris 예측 지식을 활용하는 RAG 챗봇"""
    
    def __init__(self, agent, context_builder):
        self.agent = agent
        self.context_builder = context_builder
        self.client = OpenAI()
        self.conversation_history = []
        
        print("RAG 챗봇 초기화 완료")
    
    def _get_system_prompt(self) -> str:
        knowledge_base = self.context_builder.get_knowledge_base_content()
        agent_capability = self.agent.get_capability_description()
        
        return f"""
당신은 Iris(붓꽃) 품종 분류 전문가 AI 어시스턴트다.
다음과 같은 역할을 수행한다:

1. 사용자의 붓꽃 측정 데이터를 받아 품종을 예측한다
2. 과거 예측 기록을 바탕으로 통계와 인사이트를 제공한다
3. 붓꽃 품종의 특징과 구별 방법을 설명한다
4. 예측 결과에 대한 상세한 분석과 해석을 제공한다

## 사용 가능한 도구

{agent_capability}

## 과거 예측 지식 베이스

{knowledge_base}

## 응답 가이드라인

- 친절하고 전문적인 톤을 유지한다
- 기술 용어를 사용할 때는 쉬운 설명을 덧붙인다
- 과거 예측 데이터를 활용하여 맥락있는 답변을 제공한다
- 예측 요청 시 predict_iris_species 함수를 호출한다
- 문장의 끝은 ~다로 끝낸다
""".strip()
    
    def chat(self, user_message: str) -> str:
        self.conversation_history.append({"role": "user", "content": user_message})
        
        messages = [{"role": "system", "content": self._get_system_prompt()}] + self.conversation_history
        tools = [self.agent.generate_tool_definition()]
        
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0.7
        )
        
        response_message = response.choices[0].message
        
        if response_message.tool_calls:
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                if function_name == "predict_iris_species":
                    prediction_result = self.agent.predict(function_args)
                    self.context_builder.add_prediction(prediction_result)
                    
                    self.conversation_history.append({
                        "role": "assistant",
                        "content": None,
                        "tool_calls": [tool_call]
                    })
                    
                    self.conversation_history.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": function_name,
                        "content": json.dumps(prediction_result, ensure_ascii=False)
                    })
            
            messages = [{"role": "system", "content": self._get_system_prompt()}] + self.conversation_history
            
            final_response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7
            )
            
            assistant_message = final_response.choices[0].message.content
        else:
            assistant_message = response_message.content
        
        self.conversation_history.append({"role": "assistant", "content": assistant_message})
        
        return assistant_message
    
    def generate_report(self) -> str:
        summary = self.context_builder.get_summary()
        knowledge_base = self.context_builder.get_knowledge_base_content()
        
        messages = [
            {
                "role": "system",
                "content": "당신은 데이터 분석 보고서를 작성하는 전문가다. 주어진 예측 데이터를 분석하여 경영진이 이해하기 쉬운 보고서를 작성한다. 문장의 끝은 ~다로 끝낸다."
            },
            {
                "role": "user",
                "content": f"""
다음 Iris 품종 분류 예측 데이터를 바탕으로 상세한 분석 보고서를 작성해달라:

## 요약 통계
{json.dumps(summary, indent=2, ensure_ascii=False)}

## 상세 예측 기록
{knowledge_base}

보고서에 다음 내용을 포함해달라:
1. 전체 예측 성능 요약
2. 품종별 예측 패턴 분석
3. 주요 특성별 경향성
4. 인사이트 및 권장사항
5. 향후 개선 방향
"""
            }
        ]
        
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0.7,
            max_tokens=2000
        )
        
        return response.choices[0].message.content
    
    def reset_conversation(self):
        self.conversation_history = []
        print("대화 히스토리 초기화 완료")

print("IrisRAGChatbot 클래스 정의 완료")

IrisRAGChatbot 클래스 정의 완료


---
## 5. RAG 챗봇 초기화

In [5]:
agent = IrisEnableAgent('skills/iris_agent_skill.yaml')
context_builder = PredictionContextBuilder()
chatbot = IrisRAGChatbot(agent, context_builder)

print("\n모든 컴포넌트 초기화 완료")

Enable Agent 초기화 완료: Iris Classification Agent
모델 정확도: 0.9333
Context Builder 초기화 완료
저장된 예측 로그: 10개
RAG 챗봇 초기화 완료

모든 컴포넌트 초기화 완료


---
## 6. 대화형 데모

RAG 챗봇과 대화하며 붓꽃 품종을 예측한다.

In [6]:
def chat_demo(user_input: str) -> str:
    print("\n" + "="*70)
    print(f"사용자: {user_input}")
    print("-"*70)
    
    response = chatbot.chat(user_input)
    
    print(f"챗봇: {response}")
    print("="*70)
    
    return response

In [7]:
chat_demo("안녕하세요! 붓꽃 품종 분류에 대해 알려주세요.")


사용자: 안녕하세요! 붓꽃 품종 분류에 대해 알려주세요.
----------------------------------------------------------------------
챗봇: 안녕하세요! 붓꽃 품종 분류에 대해 알려드리겠습니다. 붓꽃(Iris) 분류는 주로 세 가지 품종으로 나뉩니다: Setosa, Versicolor, Virginica입니다. 이들은 꽃받침 길이와 너비(sepal length and width), 꽃잎 길이와 너비(petal length and width)라는 네 가지 주요 특성에 기반하여 구분됩니다.

1. **Setosa**: 일반적으로 꽃잎이 짧고 넓으며, 꽃받침도 다른 두 품종에 비해 상대적으로 넓습니다. 주로 꽃잎 길이가 1.0-1.9 cm 정도로 매우 짧습니다.

2. **Versicolor**: Setosa보다는 꽃잎이 길고, Virginica보다는 짧습니다. 꽃잎 길이는 3.0-5.0 cm 정도이며, 다양한 색상을 가진 꽃잎을 가지고 있습니다.

3. **Virginica**: 가장 크고 긴 꽃잎을 가진 품종입니다. 꽃잎 길이는 보통 4.5-6.9 cm 정도로 아주 길며, 꽃받침도 다른 두 품종에 비해 더 깁니다.

붓꽃 분류는 주로 머신러닝 모델을 사용하여 자동으로 이루어지며, 각 특성의 측정 값을 입력으로 받아 품종을 예측합니다. 이 과정에서 주로 사용되는 모델은 RandomForestClassifier 같은 분류 알고리즘입니다. 이 모델은 각 특성의 중요도를 평가하여 품종을 예측하는 데 사용됩니다.

추가 질문이 있거나 붓꽃 측정 데이터를 제공해주시면, 예측을 도와드리겠습니다!


'안녕하세요! 붓꽃 품종 분류에 대해 알려드리겠습니다. 붓꽃(Iris) 분류는 주로 세 가지 품종으로 나뉩니다: Setosa, Versicolor, Virginica입니다. 이들은 꽃받침 길이와 너비(sepal length and width), 꽃잎 길이와 너비(petal length and width)라는 네 가지 주요 특성에 기반하여 구분됩니다.\n\n1. **Setosa**: 일반적으로 꽃잎이 짧고 넓으며, 꽃받침도 다른 두 품종에 비해 상대적으로 넓습니다. 주로 꽃잎 길이가 1.0-1.9 cm 정도로 매우 짧습니다.\n\n2. **Versicolor**: Setosa보다는 꽃잎이 길고, Virginica보다는 짧습니다. 꽃잎 길이는 3.0-5.0 cm 정도이며, 다양한 색상을 가진 꽃잎을 가지고 있습니다.\n\n3. **Virginica**: 가장 크고 긴 꽃잎을 가진 품종입니다. 꽃잎 길이는 보통 4.5-6.9 cm 정도로 아주 길며, 꽃받침도 다른 두 품종에 비해 더 깁니다.\n\n붓꽃 분류는 주로 머신러닝 모델을 사용하여 자동으로 이루어지며, 각 특성의 측정 값을 입력으로 받아 품종을 예측합니다. 이 과정에서 주로 사용되는 모델은 RandomForestClassifier 같은 분류 알고리즘입니다. 이 모델은 각 특성의 중요도를 평가하여 품종을 예측하는 데 사용됩니다.\n\n추가 질문이 있거나 붓꽃 측정 데이터를 제공해주시면, 예측을 도와드리겠습니다!'

In [8]:
chat_demo("꽃받침 길이 5.1cm, 꽃받침 너비 3.5cm, 꽃잎 길이 1.4cm, 꽃잎 너비 0.2cm인 붓꽃은 어떤 품종일까요?")


사용자: 꽃받침 길이 5.1cm, 꽃받침 너비 3.5cm, 꽃잎 길이 1.4cm, 꽃잎 너비 0.2cm인 붓꽃은 어떤 품종일까요?
----------------------------------------------------------------------
예측 결과 저장 완료 (총 11개)
챗봇: 입력하신 꽃받침 길이 5.1cm, 꽃받침 너비 3.5cm, 꽃잎 길이 1.4cm, 꽃잎 너비 0.2cm인 붓꽃은 **Setosa** 품종으로 예측되었습니다. 예측 신뢰도는 100%입니다, 즉 매우 높은 확신을 가지고 있습니다.

특성의 중요도를 보면, 꽃잎 길이와 꽃잎 너비가 가장 큰 영향을 미쳤습니다. 이는 Setosa 품종이 일반적으로 짧고 넓은 꽃잎을 가진다는 특징과 일치합니다.

추가 질문이 있거나 또 다른 예측을 원하시면 언제든지 말씀해 주세요!


'입력하신 꽃받침 길이 5.1cm, 꽃받침 너비 3.5cm, 꽃잎 길이 1.4cm, 꽃잎 너비 0.2cm인 붓꽃은 **Setosa** 품종으로 예측되었습니다. 예측 신뢰도는 100%입니다, 즉 매우 높은 확신을 가지고 있습니다.\n\n특성의 중요도를 보면, 꽃잎 길이와 꽃잎 너비가 가장 큰 영향을 미쳤습니다. 이는 Setosa 품종이 일반적으로 짧고 넓은 꽃잎을 가진다는 특징과 일치합니다.\n\n추가 질문이 있거나 또 다른 예측을 원하시면 언제든지 말씀해 주세요!'

In [9]:
chat_demo("이번에는 꽃받침 길이 6.3cm, 꽃받침 너비 3.3cm, 꽃잎 길이 6.0cm, 꽃잎 너비 2.5cm인 붓꽃을 예측해주세요.")


사용자: 이번에는 꽃받침 길이 6.3cm, 꽃받침 너비 3.3cm, 꽃잎 길이 6.0cm, 꽃잎 너비 2.5cm인 붓꽃을 예측해주세요.
----------------------------------------------------------------------
예측 결과 저장 완료 (총 12개)
챗봇: 입력하신 꽃받침 길이 6.3cm, 꽃받침 너비 3.3cm, 꽃잎 길이 6.0cm, 꽃잎 너비 2.5cm인 붓꽃은 **Virginica** 품종으로 예측되었습니다. 예측 신뢰도는 100%입니다, 즉 매우 높은 확신을 가지고 있습니다.

Virginica 품종은 일반적으로 가장 긴 꽃잎과 넓은 꽃잎을 갖고 있으며, 제공된 특성 값은 이 품종의 전형적인 특징과 잘 맞아떨어집니다. 이 예측에서도 꽃잎 길이와 너비가 가장 중요하게 작용했습니다.

추가 질문이나 다른 예측을 원하시면 언제든지 말씀해 주세요!


'입력하신 꽃받침 길이 6.3cm, 꽃받침 너비 3.3cm, 꽃잎 길이 6.0cm, 꽃잎 너비 2.5cm인 붓꽃은 **Virginica** 품종으로 예측되었습니다. 예측 신뢰도는 100%입니다, 즉 매우 높은 확신을 가지고 있습니다.\n\nVirginica 품종은 일반적으로 가장 긴 꽃잎과 넓은 꽃잎을 갖고 있으며, 제공된 특성 값은 이 품종의 전형적인 특징과 잘 맞아떨어집니다. 이 예측에서도 꽃잎 길이와 너비가 가장 중요하게 작용했습니다.\n\n추가 질문이나 다른 예측을 원하시면 언제든지 말씀해 주세요!'

In [10]:
chat_demo("지금까지의 예측 통계를 알려주세요.")


사용자: 지금까지의 예측 통계를 알려주세요.
----------------------------------------------------------------------
챗봇: 현재까지의 예측 통계는 다음과 같습니다:

### 전체 통계
- 총 예측 수행 횟수: 14회
- 평균 예측 신뢰도: 96.49%

### 품종별 예측 분포
- Setosa: 5회 (35.7%)
- Versicolor: 3회 (21.4%)
- Virginica: 6회 (42.9%)

### 평균 특성 값
- 꽃받침 길이: 5.89 cm
- 꽃받침 너비: 3.15 cm
- 꽃잎 길이: 4.43 cm
- 꽃잎 너비: 1.56 cm

### 최근 예측 기록 (최근 5개)
1. 예측 품종: Virginica
   - 신뢰도: 100.00%
2. 예측 품종: Setosa
   - 신뢰도: 100.00%
3. 예측 품종: Virginica
   - 신뢰도: 100.00%
4. 예측 품종: Virginica
   - 신뢰도: 100.00%
5. 예측 품종: Virginica
   - 신뢰도: 100.00%

이 통계는 최근 예측 기록과 각 품종별 예측 분포를 포함하고 있습니다. 추가적인 분석이나 다른 질문이 있으시면 언제든지 말씀해 주세요!


'현재까지의 예측 통계는 다음과 같습니다:\n\n### 전체 통계\n- 총 예측 수행 횟수: 14회\n- 평균 예측 신뢰도: 96.49%\n\n### 품종별 예측 분포\n- Setosa: 5회 (35.7%)\n- Versicolor: 3회 (21.4%)\n- Virginica: 6회 (42.9%)\n\n### 평균 특성 값\n- 꽃받침 길이: 5.89 cm\n- 꽃받침 너비: 3.15 cm\n- 꽃잎 길이: 4.43 cm\n- 꽃잎 너비: 1.56 cm\n\n### 최근 예측 기록 (최근 5개)\n1. 예측 품종: Virginica\n   - 신뢰도: 100.00%\n2. 예측 품종: Setosa\n   - 신뢰도: 100.00%\n3. 예측 품종: Virginica\n   - 신뢰도: 100.00%\n4. 예측 품종: Virginica\n   - 신뢰도: 100.00%\n5. 예측 품종: Virginica\n   - 신뢰도: 100.00%\n\n이 통계는 최근 예측 기록과 각 품종별 예측 분포를 포함하고 있습니다. 추가적인 분석이나 다른 질문이 있으시면 언제든지 말씀해 주세요!'

---
## 7. 분석 보고서 생성

In [11]:
print("분석 보고서 생성 중...")
report = chatbot.generate_report()

print("\n" + "="*70)
print("Iris 품종 분류 분석 보고서")
print("="*70)
print(report)

분석 보고서 생성 중...

Iris 품종 분류 분석 보고서
# Iris 품종 분류 예측 성능 보고서

## 1. 전체 예측 성능 요약

이번 분석의 예측 데이터는 총 12회의 예측 수행을 기반으로 하고 있다. 평균 예측 신뢰도는 96.49%로 매우 높은 수준을 나타내고 있다. 이는 모델이 Iris 품종 분류에 대해 강력한 예측 성능을 가지고 있음을 시사한다. 예측 결과는 setosa, versicolor, virginica의 세 가지 품종으로 나뉘어 있다.

## 2. 품종별 예측 패턴 분석

- **Setosa**: 총 4회 예측되었으며 전체의 33.3%를 차지하고 있다. 이는 상대적으로 낮은 비율로, 모델이 다른 두 품종보다 setosa에 대해 덜 자주 예측하는 경향이 있음을 보인다.
- **Versicolor**: 3회 예측으로 25.0%의 비율을 보인다. versicolor는 세 품종 중 예측 빈도가 가장 낮다.
- **Virginica**: 가장 높은 5회의 예측 빈도로 41.7%를 차지하고 있다. 이는 모델이 virginica를 다른 품종보다 더 자주 예측함을 나타낸다.

이러한 분포는 모델이 virginica에 대해 상대적으로 높은 인식 능력을 가지고 있음을 시사할 수 있다. 그러나 이를 통해 모델이 특정한 데이터셋에 편향되어 있을 가능성도 염두에 두어야 한다.

## 3. 주요 특성별 경향성

데이터에서 평균 특성값은 다음과 같다:
- Sepal Length: 5.82 cm
- Sepal Width: 3.12 cm
- Petal Length: 3.99 cm
- Petal Width: 1.39 cm

이러한 평균치는 일반적인 Iris 데이터셋의 특성값과 유사하여 모델이 전체적으로 안정적인 특성 인식을 하고 있음을 보여준다. 각 특성의 평균값은 모델이 예측을 수행할 때 주요하게 고려하는 기준점이 될 수 있다.

## 4. 인사이트 및 권장사항

- **인사이트**: 현재 예측에서 virginica의 높은 예측 빈도는 데이터셋의 구성이나 모델의 가중

In [12]:
report_path = 'context_store/analysis_report.md'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report)

print(f"보고서 저장 완료: {report_path}")

보고서 저장 완료: context_store/analysis_report.md


---
## 8. 시스템 현황 요약

In [13]:
summary = context_builder.get_summary()

print("="*70)
print("Enable Agent 생태계 현황")
print("="*70)
print()
print("[1] Enable Agent:")
print(f"   - Agent 이름: {agent.skill['agent_name']}")
print(f"   - 모델 타입: {agent.metadata['model_type']}")
print(f"   - 모델 정확도: {agent.metadata['accuracy']:.2%}")
print()

if summary:
    print("[2] 예측 통계:")
    print(f"   - 총 예측 횟수: {summary['total_predictions']}회")
    print(f"   - 평균 신뢰도: {summary['average_confidence']:.2%}")
    print(f"   - 품종별 분포:")
    for species, count in summary['class_distribution'].items():
        percentage = (count / summary['total_predictions']) * 100
        print(f"     * {species}: {count}회 ({percentage:.1f}%)")
    print()

print("[3] 챗봇 활동:")
print(f"   - 대화 수: {len(chatbot.conversation_history)}개")
print()
print("[4] 생성된 파일:")
print(f"   - 예측 로그: {context_builder.log_file}")
print(f"   - 통계 요약: {context_builder.summary_file}")
print(f"   - 지식 베이스: {context_builder.knowledge_base_file}")
print(f"   - 분석 보고서: context_store/analysis_report.md")
print()
print("="*70)

Enable Agent 생태계 현황

[1] Enable Agent:
   - Agent 이름: Iris Classification Agent
   - 모델 타입: RandomForestClassifier
   - 모델 정확도: 93.33%

[2] 예측 통계:
   - 총 예측 횟수: 12회
   - 평균 신뢰도: 96.49%
   - 품종별 분포:
     * setosa: 4회 (33.3%)
     * versicolor: 3회 (25.0%)
     * virginica: 5회 (41.7%)

[3] 챗봇 활동:
   - 대화 수: 12개

[4] 생성된 파일:
   - 예측 로그: context_store/prediction_logs.json
   - 통계 요약: context_store/prediction_summary.json
   - 지식 베이스: context_store/knowledge_base.txt
   - 분석 보고서: context_store/analysis_report.md

